<a href="https://colab.research.google.com/github/TheAmirHK/ReinforcementLearning_projects/blob/main/CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 36.1 MB/s eta 0:00:00


In [5]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from collections import deque

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.92
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99
        self.learning_rate = 0.0001
        self.model = self._build_model()

    def _build_model(self):
        """Build the Deep Q-Network"""
        model = Sequential([
            Dense(24, input_dim=self.state_size, activation="relu"),
            Dense(24, activation="relu"),
            Dense(self.action_size, activation="linear")
        ])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss="mse")
        return model

    def remember(self, state, action, reward, next_state, done):
        """Store experiences in replay memory"""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """Choose an action based on epsilon-greedy policy"""
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        """Train the model using a batch of experiences"""
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [6]:
env = gym.make("CartPole-v1", render_mode=None)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

episodes = 300
batch_size = 50

for e in range(episodes):
    state = env.reset()[0]
    state = np.reshape(state, [1, state_size])
    done = False
    time = 0
    while not done:
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        time += 1

        if done:
            print(f"Episode {e + 1}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2}")
            break

    agent.replay(batch_size)

state = env.reset()[0]
state = np.reshape(state, [1, state_size])
done = False
total_reward = 0

while not done:
    action = np.argmax(agent.model.predict(state, verbose=0))
    next_state, reward, done, _, _ = env.step(action)
    next_state = np.reshape(next_state, [1, state_size])
    state = next_state
    total_reward += reward
    env.render()

print(f"Total reward after training: {total_reward}")
env.close()

Episode 1/300, Score: 14, Epsilon: 1.0
Episode 2/300, Score: 30, Epsilon: 1.0
Episode 3/300, Score: 20, Epsilon: 1.0
Episode 4/300, Score: 12, Epsilon: 0.99
Episode 5/300, Score: 45, Epsilon: 0.98


KeyboardInterrupt: 